# Mortality of Switzerland and Other Countries

## 1. Human Mortality Database

Data Sources:
- [Human Mortality Database](https://www.mortality.org/)</br>
  [Short-term Mortality Fluctuations Dashboard](https://mpidr.shinyapps.io/stmortality/)</br>
  Data: https://www.mortality.org/Public/STMF/Outputs/stmf.csv

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import altair as alt


df_mortality = pd.read_csv(
    "https://www.mortality.org/Public/STMF/Outputs/stmf.csv",
    skiprows=2,
)

In [3]:
# Select countdf_mortalityf interest and only "both" sexes
# Note: Germany "DEUTNP" and "USA" have short time series
df_mortality = df_mortality[
    df_mortality["CountryCode"].isin(["CAN", "CHE", "FRATNP", "GBRTENW", "SWE"])
    & (df_mortality["Sex"] == "b")
].copy()

# Change to ISO-3166-1 ALPHA-3 codes
df_mortality["CountryCode"].replace(
    {"FRATNP": "FRA", "GBRTENW": "England & Wales"}, inplace=True
)

# Create population pro rata temporis (exposure) to ease aggregation
df_mortality = df_mortality.assign(population=lambda df: df["DTotal"] / df["RTotal"])

In [4]:
chart = (
    alt.Chart(df_mortality.reset_index())
    .transform_aggregate(
        sum_popul="sum(population)",
        sum_death="sum(DTotal)",
        groupby=["Year", "CountryCode"],
    )
    .transform_calculate(
        CDR="datum.sum_death / datum.sum_popul", Time="datetime(datum.Year, 0)"
    )
    .mark_line()
    .encode(
        x="Time:T",
        y=alt.Y("CDR:Q", scale=alt.Scale(zero=False)),
        color="CountryCode:N",
    )
    .properties(title="Crude Death Rate per Year")
    .interactive()
)
# chart.save("crude_death_rate.html")
chart

alt.Chart(...)

In [5]:
# now the same plot but with smaller data which reduces the size of html
df_mortality = (
    df_mortality.groupby(["Year", "CountryCode"])[["population", "DTotal"]]
    .sum()
    .assign(CDR=lambda x: x["DTotal"] / x["population"])
    # .filter(items=["CDR"])  # make df even smaller
    .reset_index()
    .assign(Year=lambda x: pd.to_datetime(x["Year"], format="%Y"))
)
chart = (
    alt.Chart(df_mortality)
    .mark_line()
    .encode(
        x="Year:T",
        y=alt.Y("CDR:Q", scale=alt.Scale(zero=False)),
        color="CountryCode:N",
    )
    .properties(title="Crude Death Rate per Year")
    .interactive()
)
# chart.save("crude_death_rate.html")
chart

alt.Chart(...)

## 2. Federal Statistical Office of Switzerland
We extend our analysis for Switzerland with other data sources in order to look over the past 100 years.

Data sources:
- [Todesfälle nach Monat und Sterblichkeit seit 1803](https://www.bfs.admin.ch/asset/de/px-x-0102020206_111)</br>
  BFS-Nummer px-x-0102020206_111
- [Ständige Wohnbevölkerung nach Geschlecht und Alter, 1860-2019](https://www.bfs.admin.ch/asset/de/px-x-0102030000_101)</br>
  BFS-Nummer 	px-x-0102030000_101

In [6]:
from contextlib import contextmanager
import locale

from pyaxis import pyaxis


@contextmanager
def localized(code):
    old_code, old_encoding = locale.getlocale()
    if old_code is None:
        old_code = "de_DE"
    locale.setlocale(locale.LC_ALL, code)
    yield

In [7]:
# px-x-0102020206_111
px = pyaxis.parse(
    "https://www.bfs.admin.ch/bfsstatic/dam/assets/14387168/master",
    encoding="ISO-8859-2",
)
df_death = px["DATA"]

# px-x-0102030000_101
px = pyaxis.parse(
    "https://www.bfs.admin.ch/bfsstatic/dam/assets/14819631/master",
    encoding="ISO-8859-2",
)
df_popul = px["DATA"]

In [8]:
# Preprocess deaths data
df_death = df_death[
    df_death["Demografisches Merkmal und Indikator"].str.contains("Todesfälle im")
]
df_death["Jahr"] = pd.to_numeric(df_death["Jahr"], errors="coerce")
df_death["Monat"] = df_death["Demografisches Merkmal und Indikator"].str.split().str[-1]
df_death["deaths"] = pd.to_numeric(df_death["DATA"], errors="coerce")
df_death = df_death[df_death["Jahr"] >= 1901]

with localized("de_DE.UTF-8"):
    df_death["date"] = pd.to_datetime(
        df_death["Monat"] + " " + df_death["Jahr"].astype(str), format="%B %Y"
    )

df_death.drop(columns=["Demografisches Merkmal und Indikator", "DATA"], inplace=True)
df_death.set_index("date", inplace=True)

# Preprocess population data
df_popul = df_popul[
    (df_popul["Geschlecht"] == "Geschlecht - Total")
    & (df_popul["Alter"] == "Alter - Total")
].copy()
df_popul["Jahr"] = pd.to_numeric(df_popul["Jahr"], errors="coerce")
df_popul = df_popul[df_popul["Jahr"] >= 1901]
df_popul["population"] = pd.to_numeric(df_popul["DATA"], errors="coerce").astype(float)
df_popul.drop(columns=["Alter", "Geschlecht", "DATA"], inplace=True)
df_popul.set_index("Jahr", inplace=True)

# Merging
df = pd.merge(
    df_death.groupby("Jahr").sum(),
    df_popul,
    how="inner",
    on="Jahr",
).reset_index()
df = df.assign(Year=lambda x: pd.to_datetime(x["Jahr"], format="%Y"))
df.drop(columns=["Jahr"], inplace=True)
df["CDR"] = df["deaths"] / df["population"]

In [9]:
# Compare the 2 different sources
df_compare = pd.merge(df, df_mortality.query("CountryCode == 'CHE'"), on="Year").assign(
    rel_diff=lambda x: (x["CDR_x"] - x["CDR_y"]) / x["CDR_x"]
)
print(
    "Largest relative differences (negative, positive): ",
    df_compare["rel_diff"].min(),
    df_compare["rel_diff"].max(),
)

Largest relative differences (negative, positive):  -0.005590175988630255 0.004264639219628655


This is a quite impressive match as we used two very different data sources with different time resolutions.

In [10]:
# Append year 2020 from Human Mortality data
row = (
    df_mortality.query("CountryCode == 'CHE' and Year == '2020-01-01'")
    .drop(columns="CountryCode")
    .rename(columns={"DTotal": "deaths"})
)

df = df.append(row)

In [11]:
base = alt.Chart(df).encode(
    alt.X("Year:T", scale=alt.Scale(domain=["1900-01-01", "2021-01-01"]))
)

line_A = base.mark_line(color="#5276A7").encode(
    y=alt.Y("CDR:Q", scale=alt.Scale(zero=False), axis=alt.Axis(titleColor="#5276A7"))
)

line_B = base.mark_line(color="#F18727", strokeDash=[1]).encode(
    y=alt.Y("population:Q", axis=alt.Axis(titleColor="#F18727"))
)

chart = (
    alt.layer(line_A, line_B)
    .resolve_scale(y="independent")
    .properties(title="Crude Death Rate per Year of Switzerland")
    .interactive()
)
# chart.save("swiss_mortality_chart.html")
chart

alt.LayerChart(...)